In [15]:
import os
import glob
import requests


def localize_images(markdown_file, output_dir):

    with open(markdown_file, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    id = 0
    new_lines = []
    for line in lines:
        if "](https://cdn.nlark.com" in line:
            print(line)
            start_index = line.index("](") + 2
            end_index = len(line)-2
        
            img_url =  line[start_index:end_index]
            img_name = img_url.split("#")[0].split("/")[-1]
            img_filename = f'{markdown_file.split("/")[1].split(" ")[0]}-{id}.{img_name.split(".")[-1]}'

            response = requests.get(img_url)
            with open(f'assets/{img_filename}', 'wb') as f:
                f.write(response.content)
            
            line = line.replace(img_url,f"../assets/{img_filename}")
            id+=1
        
        new_lines.append(line)


    # 写入替换好的 Markdown 文件
    replaced_file = markdown_file.replace("yuque","pages")
    with open(replaced_file, 'w', encoding='utf-8') as f:
        markdown_text = f.writelines(new_lines)


    return replaced_file



md_files = glob.glob(os.path.join("yuque", "**/*.md"), recursive=True)
md_files.sort()

# 指定图片保存的目录路径
output_directory = "assets"

# 创建保存图片的目录
os.makedirs(output_directory, exist_ok=True)

for md_file in md_files[:]:
    print(md_file)
    replaced_file = localize_images(md_file, output_directory)
    #print(f"{md_file} -> {replaced_file}")

yuque/1.1 工业决策任务介绍.md
![](https://cdn.nlark.com/yuque/0/2024/webp/12763465/1713335736991-b2e42ab8-1f65-4761-89a0-8606e58a521b.webp#averageHue=%23dbe6db&clientId=ua308efe8-6d91-4&from=paste&id=GZqWP&originHeight=500&originWidth=1000&originalType=url&ratio=1.6500000953674316&rotation=0&showTitle=false&status=done&style=none&taskId=ub5a7b383-321b-41fc-93fb-9bb94c32f45&title=)

#### ![image.png](https://cdn.nlark.com/yuque/0/2024/png/12763465/1713336473360-db13049b-34b1-4765-a288-09caf1b4bb2c.png#averageHue=%23697369&clientId=ua308efe8-6d91-4&from=paste&height=379&id=uf28b8297&originHeight=1515&originWidth=2989&originalType=binary&ratio=1.6500000953674316&rotation=0&showTitle=false&size=10379462&status=done&style=none&taskId=uf5b0a2fc-413a-49f5-89d3-f7b8cebce54&title=&width=747)

yuque/1.2 工业决策任务存在的难点.md
![image.png](https://cdn.nlark.com/yuque/0/2024/png/12763465/1715852258487-b0fdceb9-e705-4aee-87ef-fdd396c17666.png#averageHue=%23c9d2d9&clientId=ubed5705f-bedc-4&from=paste&height=236&id=